In [1]:
from pycco.parsers import *

In [2]:
params_list([tokenize('int sum(void) { return a; }')[3]])

ParserResult(position=Position(index=0), result=[], description=(type + optional '*' + identifier sep_by Token(kind=<TokenKind.SYMBOL: 4>, value=',', start=-1, end=-1)), error=None)

In [3]:
function_args(tokenize('''
(void) { return a; }
''')).result

Parser(Token(kind=<TokenKind.TYPE: 3>, value='void', start=-1, end=-1))

In [3]:
%pdb

Automatic pdb calling has been turned ON


In [4]:
print(parse_c_code('''
//yolo
int sum() { return a; }
'''))


int sum(void)
{

	return a;
}




In [7]:
    source = "int sum(int a, int b) { return a; }"
    ast = parse_c_code(source)
    assert isinstance(ast, Code)
    assert len(ast.statements) == 1

    func = ast.statements[0]
    assert isinstance(func, Function)
    # function var: "int sum"
    assert isinstance(func.var, VariableDecl)
    assert func.var.type.name == "int"
    assert func.var.type.pointer is False
    assert isinstance(func.var.name, Ident)
    assert func.var.name.name == "sum"

    # arguments
    assert len(func.args) == 2
    arg1, arg2 = func.args
    assert isinstance(arg1, Arg)
    assert arg1.type.name == "int"
    assert arg1.type.pointer is False
    assert isinstance(arg1.name, Ident)
    assert arg1.name.name == "a"

    assert isinstance(arg2, Arg)
    assert arg2.type.name == "int"
    assert arg2.type.pointer is False
    assert isinstance(arg2.name, Ident)
    assert arg2.name.name == "b"

    # Check body
    assert len(func.body) == 0  # Contains the return statement
    assert isinstance(func.ret, Return)
    assert isinstance(func.ret.value, Ident)
    assert func.ret.value.name == "a"

In [6]:
func.body

[]

In [3]:
parse_c_code('(int x)')

ValueError: too many values to unpack (expected 2)

In [5]:
tokens

[Token(kind=<TokenKind.TYPE: 3>, value='int', start=34, end=36),
 Token(kind=<TokenKind.IDENTIFIER: 1>, value='x', start=38, end=38),
 Token(kind=<TokenKind.SYMBOL: 4>, value=';', start=39, end=39),
 Token(kind=<TokenKind.EOF: 10>, value=None, start=41, end=41)]

In [5]:
eof_parser('')

ParserResult(position=Position(index=0), result=None, description=None, error=ParseError at index 0: expected EOF, found 't')

In [11]:
[*'jkl']

['j', 'k', 'l']

In [10]:
tokenize('*** ')

[Token(kind=<TokenKind.OPERATOR: 5>, value='*', start=0, end=0),
 Token(kind=<TokenKind.OPERATOR: 5>, value='*', start=1, end=1),
 Token(kind=<TokenKind.OPERATOR: 5>, value='*', start=2, end=2),
 Token(kind=<TokenKind.EOF: 10>, value=None, start=4, end=4)]

In [7]:
tokens

[Token(kind=<TokenKind.TYPE: 3>, value='float', start=0, end=4),
 Token(kind=<TokenKind.IDENTIFIER: 1>, value='pi', start=6, end=7),
 Token(kind=<TokenKind.OPERATOR: 5>, value='=', start=9, end=9),
 Token(kind=<TokenKind.NUMBER: 7>, value='3.14', start=11, end=14),
 Token(kind=<TokenKind.SYMBOL: 4>, value=';', start=15, end=15),
 Token(kind=<TokenKind.OPERATOR: 5>, value='/', start=17, end=17),
 Token(kind=<TokenKind.OPERATOR: 5>, value='/', start=18, end=18),
 Token(kind=<TokenKind.IDENTIFIER: 1>, value='Initialize', start=20, end=29),
 Token(kind=<TokenKind.IDENTIFIER: 1>, value='pi', start=31, end=32),
 Token(kind=<TokenKind.EOF: 10>, value=None, start=34, end=34)]

In [6]:
    parser = match("a") | match("b")
    stream = "c"
    position = Position(0)
    result = parser(stream, position)

[DEBUG] Calling parser '('a' | 'b')' at Position(index=0)
[DEBUG] Parser '('a' | 'b')' failed at Position(index=0) with error: ParseError at index 0: expected 'b', found 'c'


In [5]:
parser.description

"('a' | 'b')"

In [74]:
(match('5')+match(' ').optional()).sep_by(',')('5, 5')

ParserResult(index=1, result=[['5']], description=(("5" + " "?) sep_by ","))

In [77]:
args(tokenize('(int x  int y)')).result

In [70]:
args(tokenize('(void)'))
# ParserResult(index=7, result=[VariableDecl(type=Type(name='void', pointer=False), name=Ident(name='x'), semicolon=False), VariableDecl(type=Type(name='int', pointer=False), name=Ident(name='y'), semicolon=False)], description=(("(" >> (((("TYPE" + "*"?) + "IDENTIFIER") sep_by ",") | "void")) << ")"))

ParserResult(index=-1, result=None, description=(("(" >> (((("TYPE" + "*"?) + "IDENTIFIER") sep_by ",") | fn`<lambda>`)) << ")"))

In [44]:
(match(void) | variable.sep_by(Token(TokenKind.SYMBOL, ",")))(tokenize())

ParserResult(index=1, result=void, description=("void" | ((("TYPE" + "*"?) + "IDENTIFIER") sep_by ",")))

In [49]:
(variable<<Token(TokenKind.SYMBOL, ",")).until(variable)(tokenize('void x, int y')).result

[VariableDecl(type=Type(name='void', pointer=False), name=Ident(name='x'), semicolon=False),
 Token(kind=<TokenKind.SYMBOL: 4>, value=',', start=7, end=7)]

In [36]:
match(void)(tokenize('void'))

Token(kind=<TokenKind.TYPE: 3>, value='void', start=4, end=7)

In [24]:
code='''
int main(void) {
    // test case w/ multi-digit constant
    return 100;
}
'''

In [25]:
tokenize(code)

[Token(kind=<TokenKind.TYPE: 3>, value='int', start=4, end=6),
 Token(kind=<TokenKind.IDENTIFIER: 1>, value='main', start=9, end=12),
 Token(kind=<TokenKind.SYMBOL: 4>, value='(', start=10, end=10),
 Token(kind=<TokenKind.TYPE: 3>, value='void', start=14, end=17),
 Token(kind=<TokenKind.SYMBOL: 4>, value=')', start=15, end=15),
 Token(kind=<TokenKind.SYMBOL: 4>, value='{', start=17, end=17),
 Token(kind=<TokenKind.KEYWORD: 2>, value='return', start=69, end=74),
 Token(kind=<TokenKind.NUMBER: 7>, value='100', start=73, end=75),
 Token(kind=<TokenKind.SYMBOL: 4>, value=';', start=74, end=74),
 Token(kind=<TokenKind.SYMBOL: 4>, value='}', start=76, end=76),
 Token(kind=<TokenKind.EOF: 10>, value=None, start=77, end=77)]

In [26]:
function(tokenize(code))

ParserResult(index=-1, result=None, description=(((("TYPE" + "*"?) + "IDENTIFIER") + (("(" >> (((("TYPE" + "*"?) + "IDENTIFIER") sep_by ",") | "void")) << ")")) + (("{" >> (expression* + (("return" >> expression) << ";")?)) << "}")))

In [10]:
ast = parse_c_code('''
int main(void){
    int x;
    int y;
    y = 5;
    x=(1+ y);
    return 1;
}''')

In [11]:
from pprint import pprint

In [12]:
pprint(ast)

Code(statements=[Function(var=VariableDecl(type=Type(name='int', pointer=False),
                                           name=Ident(name='main'),
                                           semicolon=False),
                          args=[],
                          body=[VariableDecl(type=Type(name='int',
                                                       pointer=False),
                                             name=Ident(name='x'),
                                             semicolon=True),
                                VariableDecl(type=Type(name='int',
                                                       pointer=False),
                                             name=Ident(name='y'),
                                             semicolon=True),
                                Assign(var=Ident(name='y'),
                                       to=Number(value='5',
                                                 type=Type(name='int',
                              

In [13]:
print(ast.compile('arm'))

.section .text
.global main
main:
    PUSH {LR}    @ Save link register
    @ var decl: x
    @ var decl: y
@ Assignment
    MOV R0, #5    @ load immediate 5
    STR R0, [R1]    @ store into y
@ Assignment
    MOV R0, #1    @ load immediate 1
    PUSH {R0}
    LDR R0, =y    @ Ident y
    POP {R1}
    ADD R0, R1, R0    @ R0 = R1 + R0
    STR R0, [R1]    @ store into x
    MOV R0, #1    @ load immediate 1
    MOV PC, LR    @ Return from function


In [37]:
(expression).until(TokenKind.EOF)(tokenize('1-x')).result

[Token(kind=<TokenKind.NUMBER: 7>, value='1', start=1, end=1),
 Token(kind=<TokenKind.OPERATOR: 5>, value='-', start=2, end=2),
 Token(kind=<TokenKind.IDENTIFIER: 1>, value='x', start=3, end=3)]

In [47]:
parse_tokens(tokenize('x+1 ')).result

[Ident(name='x'),
 UnaryOp(operator='+', operand=Number(value='1', type=Type(name='int', pointer=False)))]

In [6]:
# group parens tokens
# recurse
# no parens -> precedence of binops
    # split on highest precendence binop that occurs first
    # continue while some binop token exists
    # if no binop token exists -> parse with all expressions not including binop (or temp define binop to be something else)

In [7]:
%pdb

Automatic pdb calling has been turned OFF


In [24]:
parenthesized.describe('parenthesis')

Parser(parenthesis)

In [25]:
%pdb

Automatic pdb calling has been turned OFF


In [28]:
parenthesized(tokenize('(x+1)'))

ParserResult(index=5, result=[Ident(name='x')], description=parenthesis)

In [ ]:
@parser
def binop(stream, index: int):
    left

In [3]:
non_binop_expression = number | string | ident | paren_expr

In [2]:
# 1) Our base expression (numbers, strings, ident)

base_expr = (number | string | ident | paren_expr).describe("base_expr")


# 2) The binop chain
def map_binops(parsed_list: List) -> ast.Expression:
    """
    parsed_list = [left_expr, [ (op, right_expr), (op, right_expr), ... ]]
    """
    left = parsed_list[0]
    pairs = parsed_list[1]
    for (op, right_expr) in pairs:
        left = ast.BinaryOp(left, op.value, right_expr)
    return left

binop = (
    base_expr + 
    ( any_of_enum(BinaryOperator) + base_expr ).many()
) @ map_binops

# 3) The unary_op (already defined)
def map_unary_op(elements: List[Token | ast.Expression]) -> ast.UnaryOp:
    op, operand = elements
    return ast.UnaryOp(operator=op.value, operand=operand)

unary_op = (
    any_of_enum(UnaryOperator) + expression
) @ map_unary_op

# 4) Final expression definition
expression.define(
    unary_op | binop
)

# Now 'expression' references a parser that
# - tries unary_op first, or
# - tries binop (chain of base_expr with binary operators).


Parser((((one of UnaryOperator members + None) @ map_unary_op) | ((base_expr + (one of BinaryOperator members + base_expr)*) @ map_binops)))

In [8]:
.result

RecursionError: maximum recursion depth exceeded

In [13]:
# Primary Expressions: Numbers, Strings, Identifiers, and Grouped Expressions
@parser
def primary_expression(stream: List[Token], index: int) -> ParserResult[ast.Expression]:
    return (
        number
        | string
        | ident
        | (open_paren >> expression << close_paren)  # Grouped expressions: (x + y)
    ).parse_fn(stream, index)

# Unary Operations
unary_op = (any_of_enum(UnaryOperator) + primary_expression) @ map_unary_op

# Binary Operations (Left-Associative with Precedence)
def map_binop(elements: List[ast.Node | Token]) -> ast.BinaryOp:
    left, op, right = elements
    return ast.BinaryOp(left=left, operator=op.value, right=right)

binary_operator = any_of_enum(BinaryOperator)

@parser
def binop(stream: List[Token], index: int) -> ParserResult[ast.Expression]:
    # Start by parsing the left operand (unary or primary expression)
    import pdb; pdb.set_trace()
    left_result = unary_op.parse_fn(stream, index)
    current_index = left_result.index  # Correct attribute
    left = left_result.result

    while True:
        try:
            # Parse the operator
            op_result = binary_operator.parse_fn(stream, current_index)
            operator = op_result.result
            current_index = op_result.index  # Correct attribute

            # Parse the right-hand operand
            right_result = unary_op.parse_fn(stream, current_index)
            right = right_result.result
            current_index = right_result.index  # Correct attribute

            # Combine into a BinaryOp
            left = ast.BinaryOp(left=left, operator=operator.value, right=right)

        except Exception:
            # If parsing fails, break the loop
            break

    return ParserResult(index=current_index, result=left)


# Function Calls
def map_function_call(elements: List[ast.Node | Token]) -> ast.FunctionCall:
    name, args = elements
    return ast.FunctionCall(name=name, args=args)

function_call = (
    ident
    + open_paren
    >> expression.sep_by(comma)
    << close_paren
) @ map_function_call

# Array Indexing
def map_array_index(elements: List[ast.Node]) -> ast.ArrayIndex:
    array, index = elements
    return ast.ArrayIndex(array=array, index=index)

array_index = ident + open_bracket >> expression << close_bracket @ map_array_index

# Struct Access
def map_struct_access(elements: List[ast.Node | Token]) -> ast.StructAccess:
    obj, op, field = elements
    return ast.StructAccess(obj=obj, operator=op.value, field=field)

struct_access = (
    ident
    + any_of(Symbol.DOT, Symbol.ARROW)
    + ident
) @ map_struct_access

# Comprehensive Expression Parser
@parser
def expression(stream: List[Token], index: int) -> ParserResult[ast.Expression]:
    return (
        struct_access
        | array_index
        | function_call
        | binop
        | unary_op
        | primary_expression
    ).parse_fn(stream, index)


In [14]:
# If-Else Parser
def map_if_else(elements: List[Token | ast.Node]) -> ast.IfStatement:
    condition, then_body, else_body = elements
    return ast.IfStatement(condition, then_body, else_body)

if_else = (
    match(Token(TokenKind.KEYWORD, "if"))
    >> open_paren
    >> expression
    << close_paren
    + open_bracket
    >> statement.many()
    << close_bracket
    + (match(Token(TokenKind.KEYWORD, "else")) >> open_bracket >> statement.many() << close_bracket).optional()
) @ map_if_else

In [37]:
expression(tokenize('x>0 ')).result

> /var/folders/vl/0mv20zzj0ld26z8h0ngg3wn00000gn/T/ipykernel_38709/2568700022.py(19)map_binop()
     17 
     18 def map_binop(elems):
---> 19     import pdb; pdb.set_trace()
     20     left, op, right = elems
     21     return ast.BinaryOp(left=left, operator=op.value, right=right)



ipdb>  elems


[Ident(name='x')]


ipdb>  q


In [19]:
(match(Token(TokenKind.KEYWORD, "if"))
    >> open_paren
>> expression)(tokenize('''
if (x > 0) {
    y = 1;
} else {
    y = -1;
}
'''))

RecursionError: maximum recursion depth exceeded

In [ ]:
# While Loop Parser
def map_while(elements: List[Token | ast.Node]) -> ast.WhileLoop:
    condition, body = elements
    return ast.WhileLoop(condition, body)

while_loop = (
    match(Token(TokenKind.KEYWORD, "while"))
    >> open_paren
    >> expression
    << close_paren
    + open_bracket
    >> statement.many()
    << close_bracket
) @ map_while

# For Loop Parser
def map_for(elements: List[Token | ast.Node]) -> ast.For:
    init, condition, increment, body = elements
    return ast.For(init, condition, increment, body)

for_loop = (
    match(Token(TokenKind.KEYWORD, "for"))
    >> open_paren
    >> variable_assign
    << semicolon
    + expression
    << semicolon
    + variable_assign
    << close_paren
    + open_bracket
    >> statement.many()
    << close_bracket
) @ map_for

# Function Call Parser
def map_function_call(elements: List[Token | ast.Node]) -> ast.FunctionCall:
    name, args = elements
    return ast.FunctionCall(name, args)

function_call = (
    ident
    + open_paren
    >> expression.sep_by(comma)
    << close_paren
) @ map_function_call

# Array Index Parser
def map_array_index(elements: List[Token | ast.Node]) -> ast.ArrayIndex:
    array, index = elements
    return ast.ArrayIndex(array=array, index=index)

array_index = ident + open_bracket >> expression << close_bracket @ map_array_index

# Struct Access Parser
def map_struct_access(elements: List[Token | ast.Node]) -> ast.StructAccess:
    obj, operator, field = elements
    return ast.StructAccess(obj=obj, operator=operator.value, field=field)

struct_access = (
    ident
    + any_of_enum(OtherOperator)
    .filter(lambda op: op in {OtherOperator.DOT, OtherOperator.ARROW})
    + ident
) @ map_struct_access

# Update Statement Parser
@parser
def statement(stream: List[Token], index: int) -> ParserResult[ast.Statement]:
    return (
        if_else | while_loop | for_loop | variable_decl | variable_assign | return_
    ).parse_fn(stream, index)

# Update Expression Parser
@parser
def expression(stream: List[Token], index: int) -> ParserResult[ast.Expression]:
    return (
        struct_access | array_index | function_call | unary_op | binop | ident | number | string
    ).parse_fn(stream, index)

AttributeError: module 'pycco.ast' has no attribute 'For'

In [48]:
binop(tokenize('2+"2" ')).result

BinaryOp(left=Number(value='2', type=Type(name='int', pointer=False)), operator='+', right=StringLiteral(value='2'))

In [30]:
close_bracket

Parser("}")

In [31]:
statement.many(1)(tokenize('return 2;')).result

In [32]:
(statement.until(close_bracket)@map_return)(tokenize('return 2;}')).result

Return(value=[Token(kind=<TokenKind.KEYWORD: 2>, value='return', start=6, end=11), Token(kind=<TokenKind.NUMBER: 7>, value='2', start=8, end=8), Token(kind=<TokenKind.SYMBOL: 4>, value=';', start=9, end=9)])

In [33]:
statement(tokenize('return 2;'))

ParserResult(index=-1, result=None, description=None)

In [34]:
r=function(tokenize('''
int main(char* x){
int *x=
3;
    return 2;
}''')).result

In [35]:
r.ret

Return(value=Number(value='2', type=Type(name='int', pointer=False)))

In [36]:
print(r)


int main(char* x)
{
	int* x = 3;
	return 2;
}




In [19]:
r=(((name@map_ident | variable_@map_variable_decl) << Token(TokenKind.OPERATOR, '='))+expression<<semicolon)(tokenize('int x=3;')).result

In [20]:
map_variable_assign(r)

Assign(var=VariableDecl(type=Type(name='int', pointer=False), name=Ident(name='x'), semicolon=False), to=Number(value='3', type=Type(name='int', pointer=False)))

In [ ]:
# Binary Expression
def map_binary_expression(tokens: List[Token]) -> ast.BinaryExpression:
    """
    Map tokens to a BinaryExpression.
    Assumes tokens are in the order: left operator right
    """
    left = tokens[0]
    operator = tokens[1].value
    right = tokens[2]
    return ast.BinaryExpression(
        left=ast.Ident(left.value) if left.kind == TokenKind.IDENTIFIER else left,
        operator=operator,
        right=ast.Ident(right.value) if right.kind == TokenKind.IDENTIFIER else right,
    )

expression = Parser()  # Forward declaration for expressions
binary_expression = (expression + TokenKind.OPERATOR + expression) @ map_binary_expression

# If Statement
def map_if_statement(tokens: List[Token]) -> ast.IfStatement:
    """
    Map tokens to an IfStatement.
    Assumes tokens are in the order: if (condition) { then_branch } [else { else_branch }]
    """
    condition = tokens[0]  # First token after `if` and `(`
    then_branch = tokens[1]  # Tokens within the `{}` following the condition
    else_branch = tokens[2] if len(tokens) > 2 else None  # Optional `else` branch
    return ast.IfStatement(condition=condition, then_branch=then_branch, else_branch=else_branch)

if_statement = (
    match(Token(TokenKind.KEYWORD, "if"))
    >> open_paren
    >> expression
    << close_paren
    + open_bracket
    >> variable_decl.until(close_bracket)
    << close_bracket
    + (
        match(Token(TokenKind.KEYWORD, "else"))
        >> open_bracket
        >> variable_decl.until(close_bracket)
        << close_bracket
    ).optional()
) @ map_if_statement

# While Loop
def map_while_loop(tokens: List[Token]) -> ast.WhileLoop:
    """
    Map tokens to a WhileLoop.
    Assumes tokens are in the order: while (condition) { body }
    """
    condition = tokens[0]  # First token after `while` and `(`
    body = tokens[1]  # Tokens within the `{}` following the condition
    return ast.WhileLoop(condition=condition, body=body)

while_loop = (
    match(Token(TokenKind.KEYWORD, "while"))
    >> open_paren
    >> expression
    << close_paren
    + open_bracket
    >> variable_decl.until(close_bracket)
    << close_bracket
) @ map_while_loop

# Function
def map_function(tokens: List[Token]) -> ast.Function:
    """
    Map tokens to a Function.
    Assumes tokens are in the order: return_type name(args) { body }
    """
    return_type = ast.Type(tokens[0].value)
    name = ast.Ident(tokens[1].value)
    args = [map_variable_decl(arg) for arg in tokens[2]]  # Tokens within `()`
    body = tokens[3]  # Tokens within `{}`
    return ast.Function(
        var=ast.VariableDecl(type=return_type, name=name),
        args=args,
        body=body,
    )

# Forward declaration for statement
statement = Parser()

# Function parser
def map_function(tokens: List[Token]) -> ast.Function:
    """
    Map tokens to a Function.
    Assumes tokens are in the order: return_type name(args) { body }
    """
    return_type = ast.Type(tokens[0].value)
    name = ast.Ident(tokens[1].value)
    args = [map_variable_decl(arg) for arg in tokens[2]]  # Tokens within `()`
    body = tokens[3]  # Tokens within `{}`
    return ast.Function(
        var=ast.VariableDecl(type=return_type, name=name),
        args=args,
        body=body,
    )

function = (
    type
    + name
    + (open_paren >> variable.until(comma.optional() + close_paren))
    << close_paren
    + open_bracket
    >> statement.until(close_bracket)
    << close_bracket
) @ map_function

# Define the statement parser
statement.define(variable_decl | if_statement | while_loop | binary_expression)


# Update the `_expression` placeholder to include binary_expression
expression.define(binary_expression | variable)


# Entry Point for Parsing a Program
program = function.until(TokenKind.EOF)

In [59]:
source_code = """
int* main(args) {
    int* x = 42;

    return 0;
}
"""

In [60]:
tokens = tokenize(source_code)

In [61]:
function(tokens)

ParserResult(index=-1, result=None, description=(((((("TYPE" + "*"?) + "IDENTIFIER") @ map_variable_decl) + ((("(" >> (((("TYPE" + "*"?) + "IDENTIFIER") @ map_variable_decl) sep_by ",")) << ")") @ <lambda>)) + (("{" >> (None* + ((("return" >> None) << ";") @ map_return)?)) << "}")) @ map_function))

In [12]:
variable = type + star.optional() + name
variable_decl = (variable + semicolon.optional()) @ map_variable_decl

In [13]:
variable_decl(tokenize('int main'))

AttributeError: 'str' object has no attribute 'value'

In [7]:
variable_decl(tokenize('int main'))

ParserResult(index=2, result=[Token(kind=<TokenKind.TYPE: 3>, value='int', start=3, end=5), Token(kind=<TokenKind.IDENTIFIER: 1>, value='main', start=8, end=11)], description=((("TYPE" + "*"?) + "IDENTIFIER") + ";"?))

In [9]:
parse(tokens)

ParseError: Parsing failed due to an unexpected error.
At token index 0:
int main ( args )
^
Token causing issue: int (kind: TYPE)

In [3]:
r=(variable@variable_map)(tokenize('int *main'))

NameError: name 'variable_map' is not defined

In [6]:
r.result.tokens

[Token(kind=<TokenKind.TYPE: 3>, value='int', start=3, end=5),
 Token(kind=<TokenKind.OPERATOR: 5>, value='*', start=5, end=5),
 Token(kind=<TokenKind.IDENTIFIER: 1>, value='main', start=9, end=12)]

In [4]:
(type + name)(tokenizer(('int main')))

ParserResult(index=-1, result=None, description=("TYPE" + "IDENTIFIER"))

In [6]:
(type + name + (open_paren >> variable.until(comma.optional() + close_paren)))(source_code)

ParserResult(index=-1, result=None, description=(("TYPE" + "IDENTIFIER") + ("(" >> ((("TYPE" + "*"?) + "IDENTIFIER") until (","? + ")")))))

In [3]:
type(tokenize('int'))

ParserResult(index=1, result=int, description="TYPE")

In [4]:
variable_decl(tokenize('int* x;'))

ParserResult(index=4, result=[Token(kind=<TokenKind.TYPE: 3>, value='int', start=3, end=5), Token(kind=<TokenKind.OPERATOR: 5>, value='*', start=4, end=4), Token(kind=<TokenKind.IDENTIFIER: 1>, value='x', start=6, end=6), Token(kind=<TokenKind.SYMBOL: 4>, value=';', start=7, end=7)], description=((("TYPE" + "*"?) + "IDENTIFIER") + ";"))

In [9]:
function(tokenize(source_code))

ParserResult(index=21, result=[Token(kind=<TokenKind.TYPE: 3>, value='int', start=25, end=27), Token(kind=<TokenKind.OPERATOR: 5>, value='*', start=26, end=26), Token(kind=<TokenKind.IDENTIFIER: 1>, value='x', start=28, end=28), Token(kind=<TokenKind.OPERATOR: 5>, value='=', start=30, end=30), Token(kind=<TokenKind.NUMBER: 7>, value='42', start=33, end=34), Token(kind=<TokenKind.SYMBOL: 4>, value=';', start=34, end=34), Token(kind=<TokenKind.IDENTIFIER: 1>, value='printf', start=85, end=90), Token(kind=<TokenKind.SYMBOL: 4>, value='(', start=86, end=86), Token(kind=<TokenKind.STRING_LITERAL: 6>, value='Hello, World!', start=101, end=113), Token(kind=<TokenKind.SYMBOL: 4>, value=')', start=102, end=102), Token(kind=<TokenKind.SYMBOL: 4>, value=';', start=103, end=103), Token(kind=<TokenKind.KEYWORD: 2>, value='return', start=114, end=119), Token(kind=<TokenKind.NUMBER: 7>, value='0', start=116, end=116), Token(kind=<TokenKind.SYMBOL: 4>, value=';', start=117, end=117)], description=((((

In [10]:
tokenize(source_code)

[Token(kind=<TokenKind.TYPE: 3>, value='int', start=4, end=6),
 Token(kind=<TokenKind.IDENTIFIER: 1>, value='main', start=9, end=12),
 Token(kind=<TokenKind.SYMBOL: 4>, value='(', start=10, end=10),
 Token(kind=<TokenKind.IDENTIFIER: 1>, value='args', start=14, end=17),
 Token(kind=<TokenKind.SYMBOL: 4>, value=')', start=15, end=15),
 Token(kind=<TokenKind.SYMBOL: 4>, value='{', start=17, end=17),
 Token(kind=<TokenKind.TYPE: 3>, value='int', start=25, end=27),
 Token(kind=<TokenKind.OPERATOR: 5>, value='*', start=26, end=26),
 Token(kind=<TokenKind.IDENTIFIER: 1>, value='x', start=28, end=28),
 Token(kind=<TokenKind.OPERATOR: 5>, value='=', start=30, end=30),
 Token(kind=<TokenKind.NUMBER: 7>, value='42', start=33, end=34),
 Token(kind=<TokenKind.SYMBOL: 4>, value=';', start=34, end=34),
 Token(kind=<TokenKind.IDENTIFIER: 1>, value='printf', start=85, end=90),
 Token(kind=<TokenKind.SYMBOL: 4>, value='(', start=86, end=86),
 Token(kind=<TokenKind.STRING_LITERAL: 6>, value='Hello, Worl

In [11]:
bad_code = """
int main() {
    int x = 42;
    /*
    multiline comment
    */
    printf(Hello, World!");
    return 0;
}
"""

In [12]:
tokenize(bad_code)

ValueError: Tokenization Error:
Unexpected token at line 7, column 25:

    multiline comment
    */
    printf(Hello, World!");
                        ^
    return 0;
}
Expected one of whitespace, comment, keywords, keywords, (one of {, }, (, ), ;, ,, [, ], ., -> @ create_token), (one of +, -, *, /, %, =, ==, !=, <, >, <=, >=, &&, ||, !, &, |, ^, <<, >> @ create_token), number, string literal, identifier.


In [13]:
tokenizer

Parser(one of whitespace, comment, keywords, keywords, (one of {, }, (, ), ;, ,, [, ], ., -> @ create_token), (one of +, -, *, /, %, =, ==, !=, <, >, <=, >=, &&, ||, !, &, |, ^, <<, >> @ create_token), number, string literal, identifier)